In [ ]:
pip install -q langchain-community langchain-google-genai langchain-google-genai tavily-python

Note: you may need to restart the kernel to use updated packages.


In [ ]:
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_google_genai import ChatGoogleGenerativeAI
import os
import json
from dotenv import load_dotenv

load_dotenv()
gemini_key = os.getenv("GOOGLE_API_KEY")
tavily_key = os.getenv("TAVILY_API_KEY")

In [2]:
def web_search_address(school):
    web_search_tool = TavilySearchResults(k=1)
    query = f"Địa chỉ của {school}"
    raw_results = web_search_tool.run(query)
        
    return raw_results

def extract_address_from_results(llm, results):
    prompt = f"""Bạn là Chatbot trích xuất địa chỉ.
    Nhiệm vụ của bạn là tìm tên tỉnh hoặc thành phố của Việt Nam được đề cập đầu tiên trong văn bản được cung cấp.
    Chỉ trả về tên 1 tỉnh hoặc thành phố duy nhất mà không cần thêm bất kỳ thông tin nào khác.
    {results}
    """
    address = llm.invoke(prompt)
    return address.content

In [ ]:
os.environ['TAVILY_API_KEY'] = tavily_key
os.environ['GOOGLE_API_KEY'] = gemini_key
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0)

import json
with open('raw_documents.json', 'r', encoding='utf-8') as f:
    document = json.load(f)

In [ ]:
addresses = {}
variants = [
    "TP. Hồ Chí Minh", "TP.HCM", "Tp.Hồ Chí Minh", "TP Hồ Chí Minh", "Hồ Chí Minh",
    "TP.Hồ Chí Minh", "TP. HCM", "TPHCM"
]

for school, data in document.items():
    addr = web_search_address(school)
    raw_address = addr[0]['content']
    address = extract_address_from_results(llm, raw_address)
    print(f"Địa chỉ của {school} là: {address}")
    if address in variants:
        address = "Thành phố Hồ Chí Minh"
    addresses[school] = address
    
with open('addresses.json', 'w', encoding='utf-8') as f:
    json.dump(addresses, f, ensure_ascii=False, indent=4)